In [2]:
import cv2
from keras.applications.resnet import preprocess_input as resnet_preprocess_input
from keras.applications.vgg19 import preprocess_input as vgg19_preprocess_input
from keras.models import load_model
import keras
import numpy as np 
import matplotlib.pyplot as plt 
import pandas as pd
import os
from sklearn.metrics import classification_report

In [3]:
DIR_TEST = "dataset-cropped/Test/"

In [4]:
images = []
labels = []

df = pd.read_csv("dataset-cropped/manual_ans.csv")

for tps in os.listdir(DIR_TEST):
    for paslon in os.listdir(os.path.join(DIR_TEST, tps)):
        for section in os.listdir(os.path.join(DIR_TEST, tps, paslon)):
            path = os.path.join(DIR_TEST, tps, paslon, section)

            img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
            img = cv2.resize(img, (40, 240))
            img = cv2.GaussianBlur(img, (5, 5), 0)
            # imt = cv2.medianBlur(img, 5)

            th = cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 19, 2)
            # cv2.floodFill(th, None, (0, 0), 123, )
            
            th = th / 255.0
            images.append(th)
    
    curr_tps = df[df['TPS'] == tps]
    for col in (['Paslon_1', 'Paslon_2', 'Paslon_3']):
        label_str = '{:03d}'.format(curr_tps[col].values[0])
        labels.extend([int(n) for n in label_str])

In [11]:
def evaluation(model_name, images, labels):
    model = load_model(f'models/{model_name}')
    tmp_images = np.array(images)
    tmp_labels = np.array(labels)

    tmp_images = np.repeat(tmp_images[..., np.newaxis], 3, -1)
    if model_name == 'ResNet50PreprocessInput' :
        tmp_images = resnet_preprocess_input(tmp_images)
    elif model_name == 'VGG19' :
        tmp_images = vgg19_preprocess_input(tmp_images)

    tmp_labels = keras.utils.to_categorical(tmp_labels, num_classes=10)

    loss, acc = model.evaluate(tmp_images, tmp_labels)
    print(f'Loss: {loss}')
    print(f'Accuracy: {acc}')

    y_pred = model.predict(tmp_images)
    y_pred = np.argmax(y_pred, axis=1)
    y_true = np.argmax(tmp_labels, axis=1)
    
    print(classification_report(y_true, y_pred))


RESNET50

In [12]:
evaluation('ResNet50PreprocessInput', images, labels)

51/51 [==============================] - 2s 21ms/step - loss: 0.0643 - accuracy: 0.9858
Loss: 0.06434653699398041
Accuracy: 0.9858024716377258
51/51 [==============================] - 2s 18ms/step
              precision    recall  f1-score   support

           0       0.98      1.00      0.99       528
           1       1.00      0.99      0.99       289
           2       0.99      1.00      1.00       133
           3       1.00      0.98      0.99       120
           4       1.00      1.00      1.00       107
           5       0.99      0.97      0.98        96
           6       0.97      0.97      0.97        90
           7       0.98      0.98      0.98        97
           8       0.98      0.94      0.96        84
           9       0.99      0.96      0.97        76

    accuracy                           0.99      1620
   macro avg       0.99      0.98      0.98      1620
weighted avg       0.99      0.99      0.99      1620



In [13]:
evaluation('ResNet50', images, labels)

51/51 [==============================] - 2s 21ms/step - loss: 0.0718 - accuracy: 0.9858
Loss: 0.0717562809586525
Accuracy: 0.9858024716377258
51/51 [==============================] - 2s 18ms/step
              precision    recall  f1-score   support

           0       0.99      0.99      0.99       528
           1       0.98      1.00      0.99       289
           2       0.98      0.99      0.99       133
           3       1.00      0.96      0.98       120
           4       0.99      0.98      0.99       107
           5       0.98      0.98      0.98        96
           6       0.98      0.99      0.98        90
           7       1.00      1.00      1.00        97
           8       0.98      0.96      0.97        84
           9       0.97      0.97      0.97        76

    accuracy                           0.99      1620
   macro avg       0.98      0.98      0.98      1620
weighted avg       0.99      0.99      0.99      1620



VGG19

In [14]:
evaluation('VGG19', images, labels)

51/51 [==============================] - 3s 45ms/step - loss: 0.1768 - accuracy: 0.9704
Loss: 0.17680510878562927
Accuracy: 0.970370352268219
51/51 [==============================] - 2s 30ms/step
              precision    recall  f1-score   support

           0       0.96      0.99      0.97       528
           1       0.99      0.98      0.98       289
           2       1.00      0.98      0.99       133
           3       0.98      0.97      0.97       120
           4       0.98      0.97      0.98       107
           5       0.94      0.95      0.94        96
           6       0.98      0.96      0.97        90
           7       0.98      0.95      0.96        97
           8       0.94      0.93      0.93        84
           9       0.99      0.89      0.94        76

    accuracy                           0.97      1620
   macro avg       0.97      0.96      0.96      1620
weighted avg       0.97      0.97      0.97      1620

